## Part 2a) Assigment

In [1]:
import torch
from torch import nn, Tensor
import numpy as np
import random
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchvision.io import decode_image
import os
from pathlib import Path
import pandas as pd
import seaborn as sns
from typing import List, Tuple, Dict, Optional

In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(f"Using {device} device")

Using mps device


In [3]:

dataset_paths = ["/Users/leonardomassaro/Desktop/DatasetOxford/", "/scratch.hpc/leomass/"]
correct_path = None

for path in dataset_paths:
    if os.path.exists(path) and os.path.isdir(path):
        print("Detected dataset on ", path)
        correct_path = path
if not correct_path:
    raise Exception("No dataset found")

Detected dataset on  /Users/leonardomassaro/Desktop/DatasetOxford/


In [4]:
def fix_random(seed: int) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

fix_random(42)

In [5]:
class OxfordPetDataset(Dataset):
    def __init__(self, path, transform=None) -> None:
        super().__init__()

        self.root = Path(path)
        self.transform = transform
        self.names, self.labels = self._get_names_and_labels()
        loaded_data = []
        for idx, name in enumerate(self.names):
            img_tensor = self.get_img_from_filesystem(name)
            if img_tensor == None:
                self.names.pop(idx)
                self.labels.pop(idx)
                continue
            loaded_data.append(img_tensor)

        self.data_tensor = torch.stack(loaded_data).to(device=device)
        

    def __len__(self) -> int:
        return len(self.labels)

    def __getitem__(self, idx) -> Tuple[Tensor, int]:

        label = self.labels[idx]
        img = self.data_tensor[idx]

        return img, label
    
    def get_img_from_filesystem(self, name) -> Tensor:
        img_path = self.root / "images" / f"{name}.jpg"
        img = decode_image(img_path).to(torch.float16)
        if img == None or img.shape[0] != 3:
            print("Bad channel formatting for image: ", img_path.__str__(), " skipping it...")
            return None

        if self.transform:
            img = self.transform(img)

        return img


    def get_num_classes(self) -> int:
        return max(self.labels) + 1

    def _get_names_and_labels(self) -> Tuple[List[str], List[int]]:
        names = []
        labels = []

        with open(self.root / "annotations" / "list.txt") as f:
            for line in f:
                if(line[0] == "#"):
                    continue
                name, label, _, _ = line.replace("\n", "").split(" ")
                names.append(name),
                labels.append(int(label) - 1)

        return names, labels

In [6]:
BATCH_SIZE = 64

transform_stack = transforms.Compose([
    transforms.Resize((256,256))
])

dataset = OxfordPetDataset(correct_path, transform=transform_stack)

train_ds, validation_ds, test_ds = random_split(dataset, [6/10, 2/10, 2/10])

# Create data loaders.
train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE)
validation_dataloader = DataLoader(validation_ds, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_ds, batch_size=BATCH_SIZE)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 240 extraneous bytes before marker 0xd9


Bad channel formatting for image:  /Users/leonardomassaro/Desktop/DatasetOxford/images/Egyptian_Mau_14.jpg  skipping it...
Bad channel formatting for image:  /Users/leonardomassaro/Desktop/DatasetOxford/images/Egyptian_Mau_186.jpg  skipping it...
Bad channel formatting for image:  /Users/leonardomassaro/Desktop/DatasetOxford/images/Abyssinian_5.jpg  skipping it...
Shape of X [N, C, H, W]: torch.Size([64, 3, 256, 256])
Shape of y: torch.Size([64]) torch.int64


In [ ]:

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.net_stack = nn.Sequential(
            #initial block, like resnet
            nn.LazyConv2d(32, 8),
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.MaxPool2d(3),

            #convolutional block - many convolutional blocks can be dense blocks properly concatenated
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.LazyConv2d(32, 3),

            #transition block to reduce channel dimensionality (conv has to be 1x1)
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.LazyConv2d(16, 1),
            nn.AdaptiveAvgPool2d((1,1)),

            # final block for classification
            nn.LazyBatchNorm2d(),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.LazyLinear(37)

        )

    def forward(self, x):
        #x = self.flatten(x)
        result = self.net_stack(x)
        return result

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (conv_stack): Sequential(
    (0): Conv2d(3, 1, kernel_size=(3, 3), stride=(1, 1))
    (1): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (2): Flatten(start_dim=1, end_dim=-1)
    (3): Linear(in_features=7056, out_features=1, bias=True)
  )
)


In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


RuntimeError: Input type (c10::Half) and bias type (float) should be the same